# Head to Head Performance

We have measured each pickers average performance on historic and simulated data, but what  matters the most is not how many weeks you actually survive, just that you beat the other plays in your pools. In this notebook we will pit the pickers against one another and see which performs best, and if there are different situations where one picker may be more advantageous vs another based on opponent strategies or number of players in the league. 

This notebook is to test these functions in order to use them in the streamlit app.

In [22]:
import numpy as np
import pandas as pd
import random
import itertools
from scipy.optimize import linear_sum_assignment
from collections import defaultdict
import sys
import os
sys.path.append(os.path.abspath("../scripts"))
from PickerDefinitions import Picker, BestOddsPicker, MaxOddsPicker, MaxOddsWithDecayPicker, SlidingWindowPicker, TopKOddsPicker

In [23]:
real_historic_data = pd.read_csv("../data/cleaned_odds.csv")
simulated_historic_data = pd.read_csv("../data/simulated_nfl_histories")
simulated_upcoming_season = pd.read_csv("../data/simulated_upcoming_season")

In [24]:
def play_survival_pool_season(players, season_df):
    """
    Simulates a survival pool season for a list of picker strategies.

    Parameters:
        players (list): List of picker classes (not instances) to evaluate.
        season_df (pd.DataFrame): DataFrame containing season data, including weekly games and results.

    Returns:
        tuple:
            - best_players (list): Picker class names that survived the most weeks (can be a tie).
            - best_weeks (int): Number of weeks survived by the top performer(s).
            - performances (dict): Mapping of picker class names to weeks survived.
    """
    best_weeks = 0
    best_players = []
    performances = {}
    num_each_player = {}
    for pkr in players:
        
        if pkr.__name__ not in num_each_player:
            pkr_name = pkr.__name__ + str(1)
            num_each_player[pkr.__name__] = 1
        else:
            pkr_name = pkr.__name__ + str(num_each_player[pkr.__name__] + 1)
            num_each_player[pkr.__name__] += 1
            
        this_pkr = pkr(season_df)
        this_pkr.make_season_picks()
        performance = this_pkr.evaluate_performance()
        if performance == best_weeks:
            best_players.append(pkr_name)
        elif performance > best_weeks:
            best_players = [pkr_name]
            best_weeks = performance
        performances[pkr_name] = performance

    return best_players, best_weeks, performances


def count_picker_wins(players, multi_season_df):
    """
    Evaluates picker strategies over multiple seasons and summarizes win statistics.

    Parameters:
        players (list): List of picker classes to simulate.
        multi_season_df (pd.DataFrame): Combined season data with a 'Season' column for grouping.

    Returns:
        dict:
            - 'Wins or Ties': Count of seasons each picker either won or tied for first.
            - 'Outright Wins': Count of seasons each picker won outright.
            - 'Expected Value Per Season ($1 Pool)': Picker’s expected earnings from a $1 winner(s) take all pool
    """
    win_or_tie_counts = defaultdict(int)
    outright_win_counts = defaultdict(int)
    expected_value = defaultdict(float) # this will include each strategies expected proportion of the winnings each season. 
    # If this is greater than 1/num_players, it would be considered a winning strategy

    # Group data by season for efficiency
    seaon_groups = multi_season_df.groupby('Season')
    num_seasons = len(seaon_groups)
    for _, season_df in seaon_groups:
        best_players, _, _ = play_survival_pool_season(players, season_df)

        # Increment win count for each winner (handles ties)
        num_winners = len(best_players)
        for p in best_players:
            win_or_tie_counts[p] += 1
            expected_value[p] += 1 / num_winners / num_seasons
        if num_winners == 1:
            outright_win_counts[best_players[0]] += 1

    return {'Wins or Ties': dict(win_or_tie_counts), 
            'Outright Wins': dict(outright_win_counts), 
            'Expected Value Per Season ($1 Pool)': dict(expected_value)}

def calculate_edge(expected_value_dict):
    """
    Calculates the edge the best picker has over the average expected value.
    
    Parameters:
    -----------
    expected_value_dict : dict
        Dictionary mapping picker names to their expected value (e.g., {'PickerA': 0.25, 'PickerB': 0.18, ...})
    
    Returns:
    --------
    dict with:
        - 'best_picker': name of the picker with the highest expected value
        - 'best_value': the expected value of the best picker
        - 'average_value': average expected value if all pickers were equal (1 / number of pickers)
        - 'edge': the difference between the best picker's EV and the average
    """
    num_pickers = len(expected_value_dict)
    average_value = 1 / num_pickers
    best_picker = max(expected_value_dict, key=expected_value_dict.get)
    best_value = expected_value_dict[best_picker]
    edge = best_value - average_value

    return {
        'best_picker': best_picker,
        'best_value': best_value,
        'average_value': average_value,
        'edge': edge
    }


In [25]:
all_players = [Picker, BestOddsPicker, TopKOddsPicker, MaxOddsPicker, MaxOddsWithDecayPicker, SlidingWindowPicker]

In [26]:
dup_players = [Picker, Picker, Picker, TopKOddsPicker, TopKOddsPicker]

In [27]:
# make this similar to the average weeks survived where it tests out in each dataset cleanly and for 
# different combos of playerse
one_season = real_historic_data[real_historic_data['Season'] == 2020]
play_survival_pool_season(all_players, one_season)

(['SlidingWindowPicker1'],
 17,
 {'Picker1': 1,
  'BestOddsPicker1': 12,
  'TopKOddsPicker1': 5,
  'MaxOddsPicker1': 14,
  'MaxOddsWithDecayPicker1': 14,
  'SlidingWindowPicker1': 17})

In [7]:
play_survival_pool_season(dup_players, one_season)

(['Picker3', 'TopKOddsPicker2'],
 2,
 {'Picker1': 0,
  'Picker2': 0,
  'Picker3': 2,
  'TopKOddsPicker1': 0,
  'TopKOddsPicker2': 2})

In [8]:
count_picker_wins(all_players, real_historic_data)

{'Wins or Ties': {'BestOddsPicker1': 10,
  'TopKOddsPicker1': 9,
  'MaxOddsPicker1': 10,
  'MaxOddsWithDecayPicker1': 10,
  'SlidingWindowPicker1': 14,
  'Picker1': 2},
 'Outright Wins': {'TopKOddsPicker1': 3, 'SlidingWindowPicker1': 2},
 'Expected Value Per Season ($1 Pool)': {'BestOddsPicker1': 0.1675438596491228,
  'TopKOddsPicker1': 0.22017543859649125,
  'MaxOddsPicker1': 0.1456140350877193,
  'MaxOddsWithDecayPicker1': 0.1456140350877193,
  'SlidingWindowPicker1': 0.30350877192982456,
  'Picker1': 0.017543859649122806}}

In [9]:
count_picker_wins(dup_players, real_historic_data)

{'Wins or Ties': {'TopKOddsPicker1': 13,
  'TopKOddsPicker2': 9,
  'Picker1': 3,
  'Picker3': 2},
 'Outright Wins': {'TopKOddsPicker1': 8,
  'TopKOddsPicker2': 4,
  'Picker1': 1,
  'Picker3': 1},
 'Expected Value Per Season ($1 Pool)': {'TopKOddsPicker1': 0.5263157894736843,
  'TopKOddsPicker2': 0.3157894736842105,
  'Picker1': 0.08771929824561403,
  'Picker3': 0.07017543859649122}}

In [10]:
count_picker_wins([BestOddsPicker, SlidingWindowPicker], real_historic_data)

{'Wins or Ties': {'BestOddsPicker1': 14, 'SlidingWindowPicker1': 19},
 'Outright Wins': {'SlidingWindowPicker1': 5},
 'Expected Value Per Season ($1 Pool)': {'BestOddsPicker1': 0.36842105263157887,
  'SlidingWindowPicker1': 0.631578947368421}}

Looking at Wins or Ties is helpful, but it is difficult to understand how advantageous a picker actually is against others based on it. We we combine these metrics into a single value, Expected Value Per Season ($1 Pool), that measures what portion of the pool you would expect to win when using that strategy against the other strategies. Because the true goal of a surivor pool is to maximize money won, this metric allows us to see which pickers will make you the most money. The catch with using this in practice is that you have to be able to effectively model the strategies that opponents are using. 

In [11]:
performances = count_picker_wins(all_players, simulated_historic_data)
print(performances)
EV = performances['Expected Value Per Season ($1 Pool)']
calculate_edge(EV)

{'Wins or Ties': {'Picker1': 23, 'BestOddsPicker1': 53, 'TopKOddsPicker1': 45, 'MaxOddsPicker1': 47, 'MaxOddsWithDecayPicker1': 48, 'SlidingWindowPicker1': 56}, 'Outright Wins': {'Picker1': 11, 'BestOddsPicker1': 11, 'TopKOddsPicker1': 21, 'SlidingWindowPicker1': 9}, 'Expected Value Per Season ($1 Pool)': {'Picker1': 0.12046783625730993, 'BestOddsPicker1': 0.20614035087719296, 'TopKOddsPicker1': 0.23625730994152044, 'MaxOddsPicker1': 0.11622807017543862, 'MaxOddsWithDecayPicker1': 0.12061403508771931, 'SlidingWindowPicker1': 0.20029239766081877}}


{'best_picker': 'TopKOddsPicker1',
 'best_value': 0.23625730994152044,
 'average_value': 0.16666666666666666,
 'edge': 0.06959064327485379}

In [12]:
performances = count_picker_wins(dup_players, simulated_historic_data)
print(performances)
EV = performances['Expected Value Per Season ($1 Pool)']
calculate_edge(EV)

{'Wins or Ties': {'Picker1': 18, 'TopKOddsPicker1': 45, 'TopKOddsPicker2': 46, 'Picker3': 15, 'Picker2': 21}, 'Outright Wins': {'Picker3': 8, 'TopKOddsPicker2': 33, 'TopKOddsPicker1': 30, 'Picker2': 12, 'Picker1': 10}, 'Expected Value Per Season ($1 Pool)': {'Picker1': 0.11461988304093568, 'TopKOddsPicker1': 0.3178362573099417, 'TopKOddsPicker2': 0.33684210526315816, 'Picker3': 0.09415204678362574, 'Picker2': 0.13654970760233917}}


{'best_picker': 'TopKOddsPicker2',
 'best_value': 0.33684210526315816,
 'average_value': 0.2,
 'edge': 0.13684210526315815}

In [14]:
performances = count_picker_wins(all_players, simulated_upcoming_season)
print(performances)
EV = performances['Expected Value Per Season ($1 Pool)']
calculate_edge(EV)

{'Wins or Ties': {'TopKOddsPicker1': 29, 'BestOddsPicker1': 24, 'MaxOddsPicker1': 23, 'MaxOddsWithDecayPicker1': 32, 'SlidingWindowPicker1': 33, 'Picker1': 9}, 'Outright Wins': {'TopKOddsPicker1': 17, 'Picker1': 5, 'BestOddsPicker1': 10, 'MaxOddsPicker1': 14, 'SlidingWindowPicker1': 16, 'MaxOddsWithDecayPicker1': 12}, 'Expected Value Per Season ($1 Pool)': {'TopKOddsPicker1': 0.21483333333333343, 'BestOddsPicker1': 0.14566666666666667, 'MaxOddsPicker1': 0.16650000000000004, 'MaxOddsWithDecayPicker1': 0.19233333333333338, 'SlidingWindowPicker1': 0.21733333333333338, 'Picker1': 0.06333333333333332}}


{'best_picker': 'SlidingWindowPicker1',
 'best_value': 0.21733333333333338,
 'average_value': 0.16666666666666666,
 'edge': 0.05066666666666672}

In [15]:
count_picker_wins([BestOddsPicker, SlidingWindowPicker], simulated_upcoming_season)

{'Wins or Ties': {'BestOddsPicker1': 65, 'SlidingWindowPicker1': 81},
 'Outright Wins': {'BestOddsPicker1': 19, 'SlidingWindowPicker1': 35},
 'Expected Value Per Season ($1 Pool)': {'BestOddsPicker1': 0.42000000000000026,
  'SlidingWindowPicker1': 0.5800000000000004}}